# Inspect script and extract functions and methods

In [1]:
# https://docs.python.org/3/library/inspect.html#inspect.getdoc

import inspect

In [2]:
# sig.from_function

In [3]:
inspect.getmodulename('example_module.py')
#inspect.getdoc(object)

'example_module'

# Create pytest skeleton

In [4]:
def compose_test_class_skeleton(method, params_dict_str, params_function_str):
    return (
f"""
class TestClass{method.capitalize()}():

    @pytest.mark.parametrize(
        "params,expected",
        [
            [
                {{
                    {params_dict_str}
                }},
                [expected]
                )
            ]
        ]
    )
    def test_return(self, params, expected):
        assert example_module.Arithmetic.addition({params_function_str}).equals(expected)
"""
)

In [6]:
import example_module
methods = inspect.getmembers(example_module.Arithmetic, predicate=inspect.isfunction) # predicate=inspect.ismethod
with open("test_example_module.py","w") as f:
    for method in methods:
        method_name = method[0]
        method_signature = inspect.signature(method[1])
        args = [arg for arg in method_signature.parameters.keys() if arg != 'self']
        if args:
            params_dict_str = params_function_str = ""
            
            for i in range(len(args)):
                if i < len(args)-1:
                    params_dict_str_end_format = ",\n\t\t\t\t\t"
                    params_function_str_end_format = ", "
                else:
                    params_dict_str_end_format = params_function_str_end_format = ""
                params_dict_str += f"'{args[i]}': [inputs]{params_dict_str_end_format}"
                params_function_str += f"params['{args[i]}']{params_function_str_end_format}"
                
            
            skeleton = compose_test_class_skeleton(method[0], params_dict_str, params_function_str)
            f.write(skeleton)